## Data Import Notebook - Proyect

Welcome to this first notebook. In this notebook, we will focus on connecting to the database and creating the necessary tables. Specifically, we will connect locally to a PostgreSQL instance. Afterward, we will perform some brief transformations on our dataset, which is initially in a CSV format, to prepare it for insertion into the tables we created earlier.

Before proceeding, ensure that you have already installed the necessary dependencies listed in the requirements.txt file. You can do this by running the following command:

pip install -r requirements.txt


## Set Workdir

Ensure that you already have your own .env file containing your environment variables.

In [1]:
import sys
import os
from dotenv import load_dotenv

load_dotenv()
work_dir = os.getenv('WORK_DIR')
sys.path.append(work_dir)

# libraries import

In [2]:
from src.model.models import CardioTrain
from src.database.dbconnection import getconnection
from sqlalchemy import inspect
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
from transform.TransformData import DataTransform

Using the SQLAlchemy library, connect to the database. If you encounter any issues, check that your .env file contains the correct environment variables and try again.

In [3]:
engine = getconnection()
Session = sessionmaker(bind=engine)
session = Session()

Conected successfully to database proyect!


First, this check if the candidates table exists. If a CardioTrain table is already present, it will be dropped. Therefore, be careful not to execute this code in a production environment.

In [4]:
try:
    if inspect(engine).has_table('CardioTrain'):
        CardioTrain.__table__.drop(engine)
    CardioTrain.__table__.create(engine)
    print("Table created successfully.")
except SQLAlchemyError as e:
    print(f"Error creating table: {e}")
finally:
    engine.dispose()

Table created successfully.


Now,  the original dataset is already normalize so we dont need to add things like id.

In [5]:
try:
    file = DataTransform('../data/cardio_train.csv')
    file.df.to_sql('CardioTrain', con=engine, if_exists='append', index=False)
    print("Data uploaded")

except SQLAlchemyError as e:
    print(f"Database error: {e}")

except Exception as e:
    print(f"Error: {e}")

finally:
    if hasattr(engine, 'dispose'):
        engine.dispose()

    if 'session' in locals():
        session.close()

Data uploaded


If everything was successful, a message should appear confirming that the data was successfully uploaded to our database.